# Pretrained BERT model

Use a pretrained BertModel from HuggingFace, only fit the classifier layers

In [1]:
import pandas as pd
from transformers import BertTokenizer

import re

import logging

logging.basicConfig(level=logging.WARNING)

In [2]:
df = pd.read_csv('../data/IMDB Dataset.csv')

def preprocess_imdb_raw_data(x):
    x = re.sub("<br\\s*/?>", " ", x)
    return x 

X = [preprocess_imdb_raw_data(x) for x in df['review'].values][:10]

y = df['sentiment'].apply(lambda x: int(x == 'positive')).values[:10]

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [4]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

In [5]:
from tensorflow.keras import losses

model.compile(optimizer='adam', loss=losses.binary_crossentropy)

In [6]:
tokenized_text = [tokenizer.tokenize(x) for x in X]
token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokenized_text]

In [12]:
tokenizer.tokenize(X)

AttributeError: 'list' object has no attribute 'strip'

In [7]:
model.fit(X=token_ids, y=y)

TypeError: Unrecognized keyword arguments: {'X': [[1448, 1104, 1103, 1168, 19475, 1144, 3025, 1115, 1170, 2903, 1198, 122, 16075, 2004, 1128, 112, 1325, 1129, 14111, 119, 1220, 1132, 1268, 117, 1112, 1142, 1110, 2839, 1184, 2171, 1114, 1143, 119, 1109, 1148, 1645, 1115, 4168, 1143, 1164, 16075, 1108, 1157, 28024, 1105, 8362, 2087, 2836, 7520, 4429, 1104, 4289, 117, 1134, 1383, 1107, 1268, 1121, 1103, 1937, 27157, 119, 4623, 1143, 117, 1142, 1110, 1136, 170, 1437, 1111, 1103, 7859, 21898, 1137, 189, 4060, 2386, 119, 1188, 1437, 7561, 1185, 19739, 1114, 12747, 1106, 5557, 117, 2673, 1137, 4289, 119, 2098, 1110, 16883, 117, 1107, 1103, 5263, 1329, 1104, 1103, 1937, 119, 1135, 1110, 1270, 152, 5301, 1112, 1115, 1110, 1103, 8002, 1549, 1106, 1103, 18638, 13878, 4354, 1426, 23544, 3150, 17103, 119, 1135, 7203, 2871, 1113, 24464, 1392, 117, 1126, 6700, 2237, 1104, 1103, 3315, 1187, 1155, 1103, 3652, 1138, 2525, 25036, 1105, 1339, 23120, 1116, 117, 1177, 9909, 1110, 1136, 1344, 1113, 1103, 129

In [29]:
pred = model.predict({'input_ids': np.array(token_ids[0]).reshape(-1, 1)})


In [36]:
len(token_ids[0])

396

In [32]:
pred.shape

(396, 2)

In [ ]:
pred = model.predict({'input_ids': np.array(indexed_tokens).reshape(-1, 1)})
